In [38]:
import itertools
import sys
print "Running notebook 'ks01_Data_import.ipynb'...\n"
%run 'ks01_Data_import.ipynb'
notebook_prefix = 'ks25'
image_counter = Counter()

### Reading precalculated in ks08 epistatic interactions

In [39]:
folder_with_precalculated_epistasis = files_dump_folder + 'epistasis_in_foursomes/'
natlib_filtering = True

#### Reading files

In [40]:
ep_dict = OrderedDict()
print '\nReading precalculated epistasis...\n%s\n' %folder_with_precalculated_epistasis
for segment in data:
    sys.stdout.flush()
    directory =  os.path.join(folder_with_precalculated_epistasis, segment.split('_')[0], '')
    list_of_files = [f for f in os.listdir(directory) if '.hdf' in f]
    for f in list_of_files:
        to_rstrip = '--epistasis_values.hdf'
        mutA, mutB = f.rstrip(to_rstrip).split('_')[-3], f.rstrip(to_rstrip).split('_')[-1]
        sorted_tuple = tuple(sorted([mutA, mutB], key=lambda m: m[:-1]))
        if natlib_filtering:
            if mutA in natural_unique_single_mutations['all_segments'] and mutB in natural_unique_single_mutations['all_segments']:
                ep_dict[sorted_tuple] = pd.read_hdf(directory + f, 'data')
        else:
            ep_dict[sorted_tuple] = pd.read_hdf(directory + f, 'data')       
print


Reading precalculated epistasis...
/Users/karen/HIS3InterspeciesEpistasis/Analysis/Karen/files_dump/epistasis_in_foursomes/




In [3]:
def get_ep_summary(epistasis_dictionary, function):
    floatprogress = FloatProgress(min=0, max=len(epistasis_dictionary))
    display(floatprogress)
    summary_dict = OrderedDict()
    for k,v in epistasis_dictionary.items():
        summary_dict[k] = function(v.dropna())
        floatprogress.value += 1
    return summary_dict

def get_fraction_strong(ep_series, strong_epistasis_threshold=0.4, mode='abs'):
    assert mode in ['abs', 'both_signs']
    if mode == 'abs':
        strong = ep_series[np.abs(ep_series) > strong_epistasis_threshold]
        return 1. * len(strong) / len(ep_series)
    else:
        strong_negative = ep_series[ep_series < -1 * strong_epistasis_threshold]
        strong_positive = ep_series[ep_series > strong_epistasis_threshold]
        return 1. * len(strong_negative) / len(ep_series), 1. * len(strong_positive) / len(ep_series)

In [1]:
def get_ep_summary_by_positions(epistasis_dictionary, function_for_series_summary, averaging_function):
    summary_dict = get_ep_summary(epistasis_dictionary, function_for_series_summary)
    positions_summary_dict = defaultdict(list)
    for mutA, mutB in summary_dict:
        positionA, positionB = int(mutA[:-1]), int(mutB[:-1])
        positions_summary_dict[frozenset((positionA, positionB))].append(summary_dict[mutA, mutB])
    for k,v in positions_summary_dict.items():
        positions_summary_dict[k] = averaging_function(v)
    return positions_summary_dict

In [67]:
print '''
Variables: 

- ep_dict: a dictionary, Keys are tuples (mutA, mutB) and values are pandas.Series of epistasis between mutA and mutB in all backgrounds. 
Tuples (mutA, mutB) are sorted by aa_position.

- get_ep_summary: a function that takes ep_dict as an argument and applies an arbitrary function to every pd.Series object. For example, ep_summary = get_ep_summary(ep_dict, np.mean)

- get_ep_summary_by_positions: a function that uses get_ep_summary and then averages results by position_pair. For example, get_ep_summary_by_positions(ep_dict, np.max, np.mean) will apply np.max to epistasis series and then use np.mean to average across a position pair. 

Important: natlib_filtering is currently %s. This can be changed in the cell #2.
''' %natlib_filtering


Variables: 

- ep_dict: a dictionary, Keys are tuples (mutA, mutB) and values are pandas.Series of epistasis between mutA and mutB in all backgrounds. 
Tuples (mutA, mutB) are sorted by aa_position.

- get_ep_summary: a function that takes ep_dict as an argument and applies an arbitrary function to every pd.Series object. For example, ep_summary = get_ep_summary(ep_dict, np.mean)

- get_ep_summary_by_positions: a function that uses get_ep_summary and then averages results by position_pair. For example, get_ep_summary_by_positions(ep_dict, np.max, np.mean) will apply np.max to epistasis series and then use np.mean to average across a position pair. 

Important: natlib_filtering is currently True. This can be changed in the cell #2.

